# 1. Selección de API y Definición del Problema

## API Seleccionada
- **Nombre**: Disease.sh API
- **Endpoint**: `https://disease.sh/v3/covid-19/countries`
- **Tipo de datos**: Estadísticas COVID-19 por país


## Problema de Negocio
El objetivo principal es desarrollar un sistema predictivo para anticipar brotes de COVID-19 en diferentes regiones geográficas, permitiendo a los sistemas de salud prepararse adecuadamente.


In [2]:
# Librerías para manipulación y análisis de datos
import pandas as pd
import numpy as np

# Análisis estadístico
from scipy import stats

# Librerías de visualización
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

# Peticiones API
import requests
